# English → French Translation Chatbot (Hugging Face)
This notebook builds an interactive chatbot that translates English text into French using Hugging Face's pre-trained models.  
We will:
- Install dependencies
- Load the translation model
- Add a typing animation
- Create an interactive chatbot loop
- (Optional) Build a simple Gradio web interface


In [1]:
# Install required libraries
!pip install -q "transformers[sentencepiece]" accelerate sentencepiece
#-q: short for quiet mode means reduce installation outputs so it does not prints multiple logs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.9 MB/s eta 0:00:00


## Step 1: Load the Translation Pipeline
We'll start with Hugging Face's pipeline API for quick testing using the Helsinki-NLP/opus-mt-en-fr model.


In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
# AutoModelForSeq2SeqLM : a class from transformers that automatically loads the right seq to seq language model useful for translation or summarization
# Create a translation pipeline
translator_pipe = pipeline("translation_en_to_fr", model="Helsinki-NLP/opus-mt-en-fr")
#pipline= is a high level hwlper that bundles together: A pre trained model (the brain that does the task) , A tokenizer(turn text into number the model understands and back into text after processing),any extra step(like cleaning text, handling special tokens , formatting output)
# Test translation
out = translator_pipe("Hello, world!")
print("pipeline output:", out[0]['translation_text'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

...

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


pipeline output: Bonjour, le monde !


## Step 2: Lower-Level Model Access
For more control (beam search, max length), we load the tokenizer and model manually.


In [3]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
#AutoTokenizer is the class from transformers that picks and load the correct tokemizer for the model
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
#from_pretrained = method that loads a tokenizer from a pretrained model stored online or locally
def translate_beam(text, num_beams=4, max_length=128):
#num_beams = control beam search width (more beams = potentially better but slower translations)
#max_length()= sets the maximum token length for the generated translation
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
#return_tensors = 'pt': it return tokenized output as  PyTorch tensors
#truncation=true: if the text is too long , cut it to models max length
    outputs = model.generate(**inputs, num_beams=num_beams, max_length=max_length)
    #**inputs : pass all items inputs as keyword argument to generate(unpacking dictinary)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)
#tokeinzer.decode: method to turn token IDs into a human readable string
# Quick test
print(translate_beam("How are you today?"))



Comment allez-vous aujourd'hui ?


## Step 3: Typing Animation
This function prints characters one-by-one to simulate a typing effect for chatbot responses.


In [4]:
import time

def type_print(text, delay=0.02):
    for ch in text:
        print(ch, end='', flush=True)
        time.sleep(delay)
    print()


## Step 4: Interactive Chatbot Loop
Type "exit" to quit the chatbot.


In [5]:
print("English → French Chatbot. Type 'exit' to quit.\n")

while True:
    src = input("You (English): ").strip()
    #src → Variable to store user input.
    if src.lower() in {"exit", "quit", "q"}:
        print("Bye — bonne journée! 👋")
        break
    # You can use either method:
    # fr = translator_pipe(src)[0]['translation_text']       # Simple
    fr = translate_beam(src, num_beams=4, max_length=200)    # Tunable
    #num_beams=4 → Beam search with 4 beams for better translation quality
    type_print(fr, delay=0.02)


English → French Chatbot. Type 'exit' to quit.

You (English): i love you
Je t'aime.
You (English): q
Bye — bonne journée! 👋


## Step 5 (Optional): Simple Gradio Interface
This creates a web app where you can type English text and get French translations.


In [6]:
!pip install -q gradio

import gradio as gr

def translate_for_ui(text):
    return translate_beam(text, num_beams=4, max_length=200)

iface = gr.Interface(fn=translate_for_ui, inputs="text", outputs="text",
                     title="EN → FR Chatbot",
                     description="Type English text and get French translation.")
# gr.interface : gradio main class for building simple web interfaces
iface.launch(share=True)  # share=True gives a public link in Colab


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a7cd1577ebb3213380.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Summary
We have:
- Installed and imported the Hugging Face Transformers library
- Loaded the `Helsinki-NLP/opus-mt-en-fr` model
- Created both pipeline and manual translation functions
- Added a typing animation for chatbot responses
- Built an interactive text-based chatbot
- (Optional) Created a Gradio web interface for better UX
